In [ ]:
# pinecone 데이터 임베딩
embedding = OpenAIEmbeddings(model="text-embedding-3-large")
# pinecone index 데이터 로드
index_name = 'tax-index'
# 기존에 있는 데이터를 뽑아오는 작업
vectorstore=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding,
)
# llm 셋팅
llm=ChatOpenAI(model='gpt-5')
# prompt 셋팅
prompt = hub.pull("rlm/rag-prompt")     # 허깅페이스에서 제공해줌
# retriever 셋팅
retriever=vectorstore.as_retriever(search_kwargs={'k':3})
# llm, prompt, retriever 체이닝 단계
qa_chain=RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt":prompt},
)
# 한번더 필터링 작업 후 체이닝
dictionary = ["사람을 나타내는 표현 -> 거주자"]
prompt2 = ChatPromptTemplate.from_template(f"""
사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
만약 변경이 필요없을 경우, 사용자의 질문을 변경하지 않아도 됩니다.
사전:{dictionary}
질문:{{question}}
""")
dictionary_chain = prompt2 | llm | StrOutputParser()
tex_chain = {"query": dictionary_chain} | qa_chain

In [5]:
# query="연봉 5천만원 거주자의 종합소득세는 얼마인가요? 결과만 주시죠"
query="연봉 5천만원 직장인 소득세는 얼마인가?"

In [10]:
ai_message=qa_chain.invoke({"query":query})
ai_message

{'query': '연봉 5천만원 직장인 소득세는 얼마인가?',
 'result': '과세표준이 5,000만원일 경우 산출소득세는 624만원입니다(= 84만원 + (5,000–1,400)만원×15%). 다만 연봉 5,000만원의 실제 소득세는 근로소득공제·인적공제·세액공제 반영 후의 과세표준에 따라 달라져 보통 이보다 낮습니다. 참고로 지방소득세(소득세의 10%)는 별도입니다.'}

In [12]:
new_question=dictionary_chain.invoke({"question":query})
new_question

'연봉 5천만원 거주자의 소득세는 얼마인가?'

In [13]:
ai_message=tex_chain.invoke({"question":query})
ai_message

{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가?',
 'result': '과세표준이 연봉과 같다고 가정하면, 84만원 + (5,000만원−1,400만원)×15% = 624만원입니다. 실제로는 연봉이 곧 과세표준이 아니므로 각종 공제·비과세 적용에 따라 세액이 달라질 수 있습니다.'}